In [1]:
%pip install datamule pandas tqdm


  Using cached datamule-2.4.0-py3-none-any.whl.metadata (609 bytes)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached aiohttp-3.12.15-cp313-cp313-win_amd64.whl.metadata (7.9 kB)
  Using cached aiolimiter-1.2.1-py3-none-any.whl.metadata (4.5 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached selectolax-0.4.0-cp313-cp313-win_amd64.whl.metadata (1.3 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached doc2dict-0.4.9-py3-none-any.whl.metadata (129 bytes)
  Using cached secxbrl-0.1.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached secsgml-0.3.4-py3-none-any.whl.metadata (350 bytes)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached company_fundamentals-0.0.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached flashtext-2.7-py2.py

In [2]:
from datamule import Portfolio

# Specify the directory to store downloaded filings
portfolio = Portfolio('sec_filings')


In [3]:
portfolio.download_submissions(
    filing_date=('2023-01-01', '2023-12-31'),
    submission_type=['10-K'],
    ticker=['AAPL']
)


Loading submissions
Successfully loaded 0 submissions



--- Starting query planning phase ---
Analyzing request and splitting into manageable chunks...
Fetching https://efts.sec.gov/LATEST/search-index?ciks=0000320193&forms=10-K%2C-10-K%2FA&startdt=2023-01-01&enddt=2023-12-31&from=0&size=1...
Found 1 total documents to retrieve.
Fetching https://efts.sec.gov/LATEST/search-index?ciks=0000320193&forms=10-K%2C-10-K%2FA&startdt=2023-01-01&enddt=2023-12-31&from=0&size=1...
Planning: Analyzing query: cik=0000320193, forms=10-K,-10-K/A, dates=2023-01-01 to 2023-12-31 [1 hits]
No additional forms to process with negation

--- Starting query phase ---


Querying documents [Rate: 0/s | 0 MB/s]:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching https://efts.sec.gov/LATEST/search-index?ciks=0000320193&forms=10-K%2C-10-K%2FA&startdt=2023-01-01&enddt=2023-12-31&from=0&size=100...


Querying documents [Rate: 3.0/s | 0.01 MB/s]: 100%|██████████| 1/1 [00:00<00:00,  5.58it/s]
Querying documents [Rate: 3.0/s | 0.01 MB/s]: 100%|██████████| 1/1 [00:04<00:00,  4.96s/it]


--- Query complete: 1 submissions retrieved ---

--- Streaming complete: 1 EFTS results processed ---


In [11]:
for doc in portfolio.document_type('10-K'):
    doc.parse()
    print("Metadata:", doc.data['metadata'])
    print("Document keys:", doc.data['document'].keys())
    
    for part, content in doc.data['document'].items():
        print(f"Part: {part}, type: {type(content)}")
        if isinstance(content, dict):
            for item, text in content.items():
                print(f"  Item: {item}, type: {type(text)}, len: {len(text) if isinstance(text, str) else 'N/A'}")


Metadata: {'parser': 'doc2dict', 'github': 'https://github.com/john-friedman/doc2dict', 'version': '0.4.9'}
Document keys: dict_keys([-1, 63, 269, 675, 687, 711])
Part: -1, type: <class 'dict'>
  Item: title, type: <class 'str'>, len: 12
  Item: class, type: <class 'str'>, len: 12
  Item: contents, type: <class 'dict'>, len: N/A
Part: 63, type: <class 'dict'>
  Item: title, type: <class 'str'>, len: 6
  Item: standardized_title, type: <class 'str'>, len: 5
  Item: class, type: <class 'str'>, len: 4
  Item: contents, type: <class 'dict'>, len: N/A
Part: 269, type: <class 'dict'>
  Item: title, type: <class 'str'>, len: 7
  Item: standardized_title, type: <class 'str'>, len: 6
  Item: class, type: <class 'str'>, len: 4
  Item: contents, type: <class 'dict'>, len: N/A
Part: 675, type: <class 'dict'>
  Item: title, type: <class 'str'>, len: 8
  Item: standardized_title, type: <class 'str'>, len: 7
  Item: class, type: <class 'str'>, len: 4
  Item: contents, type: <class 'dict'>, len: N/A
P

In [13]:
import re
import pandas as pd

def extract_text_from_contents(contents):
    texts = []
    if isinstance(contents, dict):
        for key, value in contents.items():
            if isinstance(value, str):
                texts.append(value)
            elif isinstance(value, dict):
                texts.extend(extract_text_from_contents(value))
            elif isinstance(value, list):
                for item in value:
                    if isinstance(item, str):
                        texts.append(item)
                    elif isinstance(item, dict):
                        texts.extend(extract_text_from_contents(item))
    return texts

data = []

for doc in portfolio.document_type('10-K'):
    doc.parse()
    meta = doc.data.get('metadata', {})
    doc_content = doc.data.get('document', {})

    for part_id, part in doc_content.items():
        contents = part.get('contents', {})
        all_texts = extract_text_from_contents(contents)
        for text in all_texts:
            if isinstance(text, str) and len(text) > 30:
                sentences = re.split(r'(?<=[.!?]) +', text)
                for sent in sentences:
                    if 30 < len(sent) < 500:
                        data.append({
                            "ticker": meta.get("ticker"),
                            "cik": meta.get("cik"),
                            "accession": meta.get("accession_number"),
                            "filing_date": meta.get("filing_date"),
                            "part_id": part_id,
                            "sentence": sent.strip()
                        })

df = pd.DataFrame(data)
df.to_csv("sec_10k_sentences.csv", index=False)
print("✅ Saved sec_10k_sentences.csv with", len(df), "sentences")



✅ Saved sec_10k_sentences.csv with 942 sentences


In [14]:
import pandas as pd
import re

df = pd.read_csv("sec_10k_sentences.csv")

def clean_sentence(sent):
    sent = re.sub(r'\s+', ' ', sent)  # collapse multiple spaces
    sent = sent.strip()
    return sent

df['sentence'] = df['sentence'].apply(clean_sentence)
df = df[df['sentence'].str.len() > 30]  # remove tiny fragments

df.to_csv("sec_10k_sentences_clean.csv", index=False)
print("✅ Saved cleaned sentences:", len(df))


✅ Saved cleaned sentences: 942


In [21]:
%pip install torch 


  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/241.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/241.3 MB 377.9 kB/s eta 0:10:38
   ---------------------------------------- 0.5/241.3 MB 377.9 kB/s eta 0:10:38
   ---------------------------------------- 0.8/241.3 MB 447.4 kB/s eta 0:08:58
   ---------

In [1]:
%pip install transformers
from transformers import pipeline
import pandas as pd
import torch

df = pd.read_csv("sec_10k_sentences_clean.csv")

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ["Strength", "Weakness", "Opportunity", "Threat"]

# Apply zero-shot model
df['swot_label'] = df['sentence'].apply(lambda x: classifier(x, candidate_labels)['labels'][0])
df.to_csv("sec_10k_sentences_swot_zero_shot.csv", index=False)
print("✅ SWOT report generated using zero-shot classification")


Note: you may need to restart the kernel to use updated packages.


d:\NLP Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
d:\NLP Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\VACHA\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see htt

✅ SWOT report generated using zero-shot classification
